# **Predicting the XRP Price using Geometric Brownian Motion for 30 days**

In [2]:
import pandas as pd
import numpy as np

In [3]:
import plotly.graph_objects as go
from plotly.offline import plot

In [4]:
# Load stock prices from CSV
file_path = "C:\\Heri\\GitHub\\Crypto\\02-Coingecko\\data-crypto.csv"
data = pd.read_csv(file_path)

In [5]:
# Assuming the CSV has a column named 'Price'
prices = data['average']  # Replace with your column name
dates = data['timestamp']  # Optional, if you want to visualize dates

In [6]:
# Calculate daily returns
daily_returns = prices.pct_change().dropna()

In [7]:
# Estimate parameters for Brownian motion
mu = daily_returns.mean()  # Mean return
sigma = daily_returns.std()  # Volatility

In [8]:
# Simulate future prices using geometric Brownian motion
def simulate_gbm(start_price, mu, sigma, time_horizon, steps):
    dt = time_horizon / steps
    price_path = [start_price]
    for _ in range(steps):
        price_path.append(price_path[-1] * np.exp((mu - 0.5 * sigma ** 2) * dt +
                                                  sigma * np.sqrt(dt) * np.random.normal()))
    return price_path

In [9]:
# Parameters for simulation
start_price = prices.iloc[-1]  # Start from the last known price
time_horizon = 365  # Days
steps = 3650  # Simulation steps

In [10]:
# Run simulation
simulated_prices = simulate_gbm(start_price, mu, sigma, time_horizon, steps)

In [11]:
# Create the plot
fig = go.Figure()

# Add the simulated prices line
fig.add_trace(go.Scatter(
    x=list(range(len(simulated_prices))),
    y=simulated_prices,
    mode='lines',
    name='Simulated Prices',
    line=dict(color='#206583', width=2)
))

# Update layout for aesthetics
fig.update_layout(
    title={
        'text': 'Average Crypto Market Price Prediction using GBM for 1 Year',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Time Step',
    yaxis_title='Price',
    font=dict(size=14),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [12]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='Average Crypto Market Price Prediction.html')

'Average Crypto Market Price Prediction.html'

In [13]:
# Run simulation 10 times
simulations = []

simulations.append(simulated_prices)

for simulation in range(14):
    simulated_prices = simulate_gbm(start_price, mu, sigma, time_horizon, steps)
    simulations.append(simulated_prices)

In [16]:
# Create the plot
fig = go.Figure()

# Define a list of colors
colors = ['#0D3239', '#1A5465', '#206583', '#3487AB', '#3487AB', '#0D3239', '#113C46', '#184F5F', '#1D5B72', '#1E607A', '#266F8E', '#3182A5', '#3487AB', '#3487AB', '#3487AB']

# Add each simulated prices line with different colors
for i, prices in enumerate(simulations):
    fig.add_trace(go.Scatter(
        x=list(range(len(prices))),
        y=prices,
        mode='lines',
        name=f'Simulation {i+1}',
        line=dict(color=colors[i % len(colors)], width=1)
    ))

# Update layout for aesthetics
fig.update_layout(
    title={
        'text': 'Average Crypto Market Price Prediction using GBM for 1 Year',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Time Step',
    yaxis_title='Price',
    font=dict(size=14),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [17]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='Average Crypto Market Price Multiple Prediction using GBM for 1 Year.html')

'Average Crypto Market Price Multiple Prediction using GBM for 1 Year.html'